In [39]:
import datetime as datetime
import numpy as np
import pandas as pd
import os
import warnings
import time

from Data import *
from Speed_Fit import *
from Variance_Fit import *
from Scorer import *

warnings.filterwarnings("ignore")

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [19]:
test = Data('./SFF_16_20.csv')
test.filter(race_type=['b'])
test.match('C:\\Users\\mvopa\\Desktop\\Betfair\\data\\UK\\New\\dwbfpricesukwin')

62237 races before filtering
1817 races after filtering
1203 races after matching


In [29]:
fit = Speed_Fit(test.base)
fit.linear_race_fit()
fit.linear_horse_fit()

0.554 R^2 Race General features 
0.6097 R^2 All
0.0557 Horse Specific R^2 difference


In [37]:
var_fit = Variance_Fit(fit.final)
var_fit.associate_constant_variance(0.5)

In [42]:
simulation = Scorer(var_fit.base)
simulation.simulate(10000)
simulation.score()

## Grid search optimising constant variance

In [49]:
storage = {}
for i in [np.round(j,decimals=2) for j in np.linspace(0.1,0.8,15)]:
    var_fit = Variance_Fit(fit.final)
    var_fit.associate_constant_variance(i)
    
    simulation = Scorer(var_fit.base)
    simulation.simulate(10000)
    simulation.score()
    
    storage[i] = [simulation.score,simulation.score2]

In [50]:
storage

{0.1: [38.14309095759053, 340.16863114883057],
 0.15: [38.041341812181244, 231.9547678699586],
 0.2: [38.075194700709446, 178.9706748096327],
 0.25: [38.075058677695274, 152.44529367638646],
 0.3: [38.07393835916394, 138.57563888672814],
 0.35: [38.08421890450942, 131.30801391284348],
 0.4: [38.080352156035154, 127.55595499422125],
 0.45: [38.08368052951052, 125.61470637094968],
 0.5: [38.08254841773214, 124.83097431209063],
 0.55: [38.09890803071367, 124.60003798730466],
 0.6: [38.06976134716326, 125.10188103104738],
 0.65: [38.086529959778325, 125.78331734949592],
 0.7: [38.08424119582571, 126.36038766578241],
 0.75: [38.07329866087385, 127.22904113435476],
 0.8: [38.08494260424557, 127.99987850059537]}